# 03. Extract features from dataset by using MobileNet(pre-trained model)
* we'll made **`X_train_reshaped_XXXXXX.npy`** and **`X_test_reshaped_XXXXXX.npy`** through this code.
* **`Before run this file, Please check this`**:
    * 01_video-to-numpy-save.ipynb
    * 02_create-numpy-datasets_training-test.ipynb
* **`Are those files exist on there?`** Those files were made by 01~02 ipynb files.
    * 02_data_training_Numpy_210512.pickle
    * 02_data_test_Numpy_210512.pickle

# Imports 

In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import random

# 03-A. Load datasets array(.npy) : trainning set & test set
* shape : `(# of Video file, # of frame img, img height, img width, RGB)`

In [2]:
data_training_ar=np.load('D:/datasets/AllVideo_numpy_list_pickle/02_data_training_Numpy_210512.npy') 
data_training_ar.shape #> (2878, 30, 160, 160, 3) 

(2878, 30, 160, 160, 3)

In [3]:
data_test_ar=np.load('D:/datasets/AllVideo_numpy_list_pickle/02_data_test_Numpy_210512.npy') 
data_test_ar.shape #> (720, 30, 160, 160, 3)

(720, 30, 160, 160, 3)

# 03-B. Reshape data
* `(# of Video file, # of frame img, img height, img width, RGB)` to `((# of Video file)*(# of frame img), img height, img width, RGB)`

In [4]:
data_training_ar=data_training_ar.reshape(data_training_ar.shape[0]*30, 160, 160, 3) 
data_training_ar.shape #> (86340, 160, 160, 3)

(86340, 160, 160, 3)

In [5]:
data_test_ar=data_test_ar.reshape(data_test_ar.shape[0]*30, 160, 160, 3) 
data_test_ar.shape #> (21600, 160, 160, 3)

(21600, 160, 160, 3)

# 03-C. Create Base Model : MobileNet

In [6]:
base_model=keras.applications.mobilenet.MobileNet(input_shape=(160, 160, 3),
                                                  include_top=False,
                                                  weights='imagenet', classes=2)

In [7]:
base_model.summary() #> (None, 5, 5, 1024)

Model: "mobilenet_1.00_160"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 80, 80, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 80, 80, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 80, 80, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 80, 80, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 80, 80, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 80, 80, 32) 

# 03-D. Predict(Extract features) : Insert datasets to MobileNet base_model
* create **`X_train`**, **`X_test`**
* **output shape** : **`((# of Video file)*(# of frame img), 5, 5, 1024)`**

In [8]:
np.random.seed(42)

In [9]:
X_train=base_model.predict(data_training_ar)
X_train.shape #> (86340, 5, 5, 1024)

(86340, 5, 5, 1024)

In [10]:
X_test=base_model.predict(data_test_ar)
X_test.shape #> (21600, 5, 5, 1024)

(21600, 5, 5, 1024)

# 03-E. Reshape predict result to insert LSTM
* **Output Shape** : **`(# of Video File, # of frame img, 5*5*1024)`**

In [13]:
X_train_reshaped = X_train.reshape(int(X_train.shape[0]/30), 30, 5*5*1024) #> (2878, 30, 25600) ndarray

In [14]:
X_test_reshaped = X_test.reshape(int(X_test.shape[0]/30), 30, 5*5*1024) #> (720, 30, 25600) ndarray

In [15]:
X_train_reshaped.shape, X_test_reshaped.shape

((2878, 30, 25600), (720, 30, 25600))

# 03-F. Save reshaped result file(.npy)
* **`MobileNet_x_train_reshaped_210512.npy`**, **`MobileNet_x_test_reshaped_210512.npy`**

In [16]:
np.save('D:/datasets/AllVideo_numpy_list_pickle/MobileNet_x_train_reshaped_210512.npy', X_train_reshaped)

In [17]:
np.save('D:/datasets/AllVideo_numpy_list_pickle/MobileNet_x_test_reshaped_210512.npy', X_test_reshaped)